## Setting Up Environment

In [1]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers accelerate

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install evaluate

Note: you may need to restart the kernel to use updated packages.


In [1]:
# import numpy, matplotlib.pylot and panda
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import librosa
import torch

from scipy.signal import butter, filtfilt

In [2]:
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

True
Using device: cuda


In [3]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4070 Laptop GPU'

## Data Collection

In [2]:
dataset = pd.read_csv('cv-valid-test.csv')
dataset

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-test/sample-000000.mp3,without the dataset the article is useless,1,0,NaN,NaN,NaN,NaN
1,cv-valid-test/sample-000001.mp3,i've got to go to him,1,0,twenties,male,NaN,NaN
2,cv-valid-test/sample-000002.mp3,and you know it,1,0,NaN,NaN,NaN,NaN
3,cv-valid-test/sample-000003.mp3,down below in the darkness were hundreds of pe...,4,0,twenties,male,us,NaN
4,cv-valid-test/sample-000004.mp3,hold your nose to keep the smell from disablin...,2,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3990,cv-valid-test/sample-003990.mp3,the old man opened his cape and the boy was st...,1,0,NaN,NaN,NaN,NaN
3991,cv-valid-test/sample-003991.mp3,in alchemy it's called the soul of the world,2,1,NaN,NaN,NaN,NaN
3992,cv-valid-test/sample-003992.mp3,at that point in their lives everything is cle...,3,0,NaN,NaN,NaN,NaN
3993,cv-valid-test/sample-003993.mp3,he told them all to be seated,3,0,NaN,NaN,NaN,NaN


## Data Preprocessing

In [3]:
dataset = dataset[["filename", "text"]]
dataset

,filename,text
0,cv-valid-test/sample-000000.mp3,without the dataset the article is useless
1,cv-valid-test/sample-000001.mp3,i've got to go to him
2,cv-valid-test/sample-000002.mp3,and you know it
3,cv-valid-test/sample-000003.mp3,down below in the darkness were hundreds of pe...
4,cv-valid-test/sample-000004.mp3,hold your nose to keep the smell from disablin...
...,...,...
3990,cv-valid-test/sample-003990.mp3,the old man opened his cape and the boy was st...
3991,cv-valid-test/sample-003991.mp3,in alchemy it's called the soul of the world
3992,cv-valid-test/sample-003992.mp3,at that point in their lives everything is cle...
3993,cv-valid-test/sample-003993.mp3,he told them all to be seated


In [4]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def prepare_dataset(batch,processor=processor):
     # load audio sample FROM PATH with specified sampling rate   
    audio_array, sampling_rate = librosa.load(batch["filename"], sr=16000, mono=True)

    # compute log-Mel input features from input audio array 
    batch["input_features"] = processor.feature_extractor(audio_array, sampling_rate=sampling_rate).input_features[0]

    # encode target text to label ids 
    batch["labels"] = processor.tokenizer(batch["text"]).input_ids
    return batch

In [6]:
dataset=dataset.apply(prepare_dataset,axis=1)

In [7]:
dataset = dataset[['input_features', 'labels']]
dataset

,input_features,labels
0,"[[-0.6730652, -0.6730652, -0.6730652, -0.67306...","[50258, 50259, 50359, 50363, 11820, 346, 264, ..."
1,"[[-0.7859349, -0.7859349, -0.7859349, -0.73237...","[50258, 50259, 50359, 50363, 72, 600, 658, 281..."
2,"[[-0.79748297, -0.79748297, -0.79748297, -0.46...","[50258, 50259, 50359, 50363, 474, 291, 458, 30..."
3,"[[-1.0352554, -1.0352554, -1.0352554, -1.03525...","[50258, 50259, 50359, 50363, 5093, 2507, 294, ..."
4,"[[-0.57676864, -0.57676864, -0.57676864, -0.44...","[50258, 50259, 50359, 50363, 4104, 428, 6690, ..."
...,...,...
3990,"[[-0.9301528, -0.9301528, -0.9301528, -0.93015...","[50258, 50259, 50359, 50363, 3322, 1331, 587, ..."
3991,"[[-0.7404628, -0.7404628, -0.7404628, -0.74046...","[50258, 50259, 50359, 50363, 259, 419, 339, 36..."
3992,"[[-0.63220394, -0.63220394, -0.63220394, -0.63...","[50258, 50259, 50359, 50363, 267, 300, 935, 29..."
3993,"[[-0.63407266, -0.63407266, -0.63407266, -0.63...","[50258, 50259, 50359, 50363, 675, 1907, 552, 4..."


### Splitting into Training and Test Datasets

In [8]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2)

In [9]:
train = train.reset_index(drop=True)
train

,input_features,labels
0,"[[-0.7396982, -0.7396982, -0.7396982, -0.56966...","[50258, 50259, 50359, 50363, 675, 1415, 281, 9..."
1,"[[-1.1065807, -1.1065807, -1.1065807, -0.47670...","[50258, 50259, 50359, 50363, 72, 390, 6359, 49..."
2,"[[-1.0652044, -1.0652044, -1.0652044, -0.87513...","[50258, 50259, 50359, 50363, 351, 291, 722, 42..."
3,"[[-1.2244408, -1.2244408, -1.2244408, -1.22444...","[50258, 50259, 50359, 50363, 72, 390, 13856, 4..."
4,"[[-0.545365, -0.545365, -0.545365, -0.2958833,...","[50258, 50259, 50359, 50363, 19096, 5800, 415,..."
...,...,...
3191,"[[-0.53368616, -0.53368616, -0.53368616, -0.53...","[50258, 50259, 50359, 50363, 13162, 645, 1706,..."
3192,"[[-1.0352554, -1.0352554, -1.0352554, -1.03525...","[50258, 50259, 50359, 50363, 5093, 2507, 294, ..."
3193,"[[-0.6904956, -0.6904956, -0.6904956, -0.62819...","[50258, 50259, 50359, 50363, 675, 14226, 264, ..."
3194,"[[-0.60460687, -0.60460687, -0.60460687, -0.45...","[50258, 50259, 50359, 50363, 474, 370, 7228, 2..."


In [10]:
test = test.reset_index(drop=True)
test

,input_features,labels
0,"[[-1.0096068, -1.0096068, -1.0096068, -1.00960...","[50258, 50259, 50359, 50363, 336, 436, 519, 46..."
1,"[[-1.0501065, -1.0501065, -1.0501065, -0.83016...","[50258, 50259, 50359, 50363, 72, 5298, 291, 64..."
2,"[[-0.53549445, -0.53549445, -0.53549445, -0.53...","[50258, 50259, 50359, 50363, 49800, 18185, 337..."
3,"[[-0.5928992, -0.5928992, -0.5928992, -0.59289...","[50258, 50259, 50359, 50363, 39380, 570, 300, ..."
4,"[[-0.686262, -0.686262, -0.64569473, -0.234107...","[50258, 50259, 50359, 50363, 5616, 603, 764, 4..."
...,...,...
794,"[[-0.8432269, -0.8432269, -0.8432269, -0.59362...","[50258, 50259, 50359, 50363, 18822, 291, 366, ..."
795,"[[-1.0786912, -1.0786912, -1.0786912, -0.88296...","[50258, 50259, 50359, 50363, 15456, 390, 588, ..."
796,"[[-1.2304549, -1.2304549, -1.2304549, -1.23045...","[50258, 50259, 50359, 50363, 270, 1633, 362, 1..."
797,"[[-0.772483, -0.772483, -0.6983535, -0.0572773...","[50258, 50259, 50359, 50363, 270, 534, 13416, ..."


In [11]:
validation, benchmark = train_test_split(test, test_size=0.1)

In [12]:
benchmark = benchmark.reset_index(drop=True)
benchmark

,input_features,labels
0,"[[-1.1315699, -1.1315699, -1.1315699, -1.13156...","[50258, 50259, 50359, 50363, 3322, 9488, 2610,..."
1,"[[-0.74414, -0.74414, -0.74414, -0.74414, -0.5...","[50258, 50259, 50359, 50363, 1902, 292, 1152, ..."
2,"[[-0.8293656, -0.8293656, -0.8293656, -0.59377...","[50258, 50259, 50359, 50363, 4286, 360, 561, 8..."
3,"[[-0.7068156, -0.7068156, -0.7068156, -0.44695...","[50258, 50259, 50359, 50363, 3322, 3237, 3031,..."
4,"[[-0.87031555, -0.87031555, -0.87031555, -0.87...","[50258, 50259, 50359, 50363, 306, 12674, 6869,..."
...,...,...
75,"[[-0.9255625, -0.9255625, -0.9255625, -0.92556...","[50258, 50259, 50359, 50363, 3322, 3237, 5694,..."
76,"[[-0.82639194, -0.82639194, -0.82639194, -0.82...","[50258, 50259, 50359, 50363, 266, 527, 636, 28..."
77,"[[-0.73914635, -0.73914635, -0.73914635, -0.57...","[50258, 50259, 50359, 50363, 71, 3145, 732, 29..."
78,"[[-0.5983442, -0.5983442, -0.5983442, 0.034376...","[50258, 50259, 50359, 50363, 20579, 456, 415, ..."


In [13]:
train = train.T
train

,0,1,2,3,4,5,6,7,8,9,...,3186,3187,3188,3189,3190,3191,3192,3193,3194,3195
input_features,"[[-0.7396982, -0.7396982, -0.7396982, -0.56966...","[[-1.1065807, -1.1065807, -1.1065807, -0.47670...","[[-1.0652044, -1.0652044, -1.0652044, -0.87513...","[[-1.2244408, -1.2244408, -1.2244408, -1.22444...","[[-0.545365, -0.545365, -0.545365, -0.2958833,...","[[-0.9114121, -0.9114121, -0.9114121, 0.167829...","[[-0.8647183, -0.8647183, -0.8647183, -0.86471...","[[-0.57422614, -0.57422614, -0.57422614, 0.134...","[[-0.79151523, -0.79151523, -0.79151523, -0.60...","[[-0.56270766, -0.56270766, -0.56270766, -0.56...",...,"[[-1.2383907, -1.2383907, -1.2224436, -1.17380...","[[-1.0518277, -1.0518277, -1.0335014, -0.55342...","[[-0.80575323, -0.80575323, -0.80575323, -0.80...","[[-0.6096194, -0.6096194, -0.6096194, -0.60961...","[[-0.6775445, -0.6775445, -0.6775445, -0.56451...","[[-0.53368616, -0.53368616, -0.53368616, -0.53...","[[-1.0352554, -1.0352554, -1.0352554, -1.03525...","[[-0.6904956, -0.6904956, -0.6904956, -0.62819...","[[-0.60460687, -0.60460687, -0.60460687, -0.45...","[[-0.76840913, -0.76840913, -0.76840913, -0.56..."
labels,"[50258, 50259, 50359, 50363, 675, 1415, 281, 9...","[50258, 50259, 50359, 50363, 72, 390, 6359, 49...","[50258, 50259, 50359, 50363, 351, 291, 722, 42...","[50258, 50259, 50359, 50363, 72, 390, 13856, 4...","[50258, 50259, 50359, 50363, 19096, 5800, 415,...","[50258, 50259, 50359, 50363, 539, 437, 360, 29...","[50258, 50259, 50359, 50363, 1445, 436, 434, 5...","[50258, 50259, 50359, 50363, 11176, 3144, 2203...","[50258, 50259, 50359, 50363, 270, 311, 257, 22...","[50258, 50259, 50359, 50363, 270, 2586, 295, 2...",...,"[50258, 50259, 50359, 50363, 72, 727, 978, 199...","[50258, 50259, 50359, 50363, 3322, 3440, 390, ...","[50258, 50259, 50359, 50363, 69, 1023, 561, 18...","[50258, 50259, 50359, 50363, 11176, 390, 264, ...","[50258, 50259, 50359, 50363, 72, 1604, 577, 74...","[50258, 50259, 50359, 50363, 13162, 645, 1706,...","[50258, 50259, 50359, 50363, 5093, 2507, 294, ...","[50258, 50259, 50359, 50363, 675, 14226, 264, ...","[50258, 50259, 50359, 50363, 474, 370, 7228, 2...","[50258, 50259, 50359, 50363, 71, 1680, 279, 70..."


In [14]:
test = test.T
test

,0,1,2,3,4,5,6,7,8,9,...,789,790,791,792,793,794,795,796,797,798
input_features,"[[-1.0096068, -1.0096068, -1.0096068, -1.00960...","[[-1.0501065, -1.0501065, -1.0501065, -0.83016...","[[-0.53549445, -0.53549445, -0.53549445, -0.53...","[[-0.5928992, -0.5928992, -0.5928992, -0.59289...","[[-0.686262, -0.686262, -0.64569473, -0.234107...","[[-0.6876404, -0.6876404, -0.6876404, -0.68764...","[[-0.8542875, -0.8542875, -0.8542875, -0.85428...","[[-0.8479985, -0.8479985, -0.8479985, -0.84799...","[[-0.8816, -0.8816, -0.8816, -0.359483, -0.666...","[[-0.8577628, -0.8577628, -0.8577628, -0.74194...",...,"[[-0.97213197, -0.97213197, -0.97213197, -0.97...","[[-1.1351261, -1.1351261, -0.9823216, -0.26160...","[[-0.64631915, -0.64631915, -0.64631915, -0.56...","[[-0.96042037, -0.96042037, -0.9181441, -0.564...","[[-0.42516565, -0.42516565, -0.42516565, -0.08...","[[-0.8432269, -0.8432269, -0.8432269, -0.59362...","[[-1.0786912, -1.0786912, -1.0786912, -0.88296...","[[-1.2304549, -1.2304549, -1.2304549, -1.23045...","[[-0.772483, -0.772483, -0.6983535, -0.0572773...","[[-0.8870206, -0.8870206, -0.8870206, -0.88702..."
labels,"[50258, 50259, 50359, 50363, 336, 436, 519, 46...","[50258, 50259, 50359, 50363, 72, 5298, 291, 64...","[50258, 50259, 50359, 50363, 49800, 18185, 337...","[50258, 50259, 50359, 50363, 39380, 570, 300, ...","[50258, 50259, 50359, 50363, 5616, 603, 764, 4...","[50258, 50259, 50359, 50363, 474, 1310, 309, 2...","[50258, 50259, 50359, 50363, 13162, 434, 1009,...","[50258, 50259, 50359, 50363, 270, 311, 47976, ...","[50258, 50259, 50359, 50363, 259, 729, 1708, 5...","[50258, 50259, 50359, 50363, 5479, 360, 291, 5...",...,"[50258, 50259, 50359, 50363, 270, 6576, 281, 7...","[50258, 50259, 50359, 50363, 72, 3428, 2573, 5...","[50258, 50259, 50359, 50363, 3322, 1194, 390, ...","[50258, 50259, 50359, 50363, 71, 521, 3953, 93...","[50258, 50259, 50359, 50363, 474, 415, 2067, 3...","[50258, 50259, 50359, 50363, 18822, 291, 366, ...","[50258, 50259, 50359, 50363, 15456, 390, 588, ...","[50258, 50259, 50359, 50363, 270, 1633, 362, 1...","[50258, 50259, 50359, 50363, 270, 534, 13416, ...","[50258, 50259, 50359, 50363, 13966, 380, 291, ..."


In [15]:
benchmark = benchmark.T
benchmark

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
input_features,"[[-1.1315699, -1.1315699, -1.1315699, -1.13156...","[[-0.74414, -0.74414, -0.74414, -0.74414, -0.5...","[[-0.8293656, -0.8293656, -0.8293656, -0.59377...","[[-0.7068156, -0.7068156, -0.7068156, -0.44695...","[[-0.87031555, -0.87031555, -0.87031555, -0.87...","[[-1.3740661, -1.3740661, -1.277633, -0.649837...","[[-0.77064395, -0.77064395, -0.77064395, -0.24...","[[-0.99318385, -0.99318385, -0.99318385, -0.61...","[[-0.87945354, -0.87945354, -0.87945354, -0.43...","[[-0.86943305, -0.86943305, -0.86943305, -0.86...",...,"[[-0.84461737, -0.84461737, -0.84461737, -0.20...","[[-0.8953326, -0.8953326, -0.8953326, -0.89533...","[[-0.7832053, -0.7832053, -0.5498072, 0.148869...","[[-0.55103076, -0.55103076, -0.55103076, -0.55...","[[-0.5610559, -0.5610559, -0.5610559, -0.11793...","[[-0.9255625, -0.9255625, -0.9255625, -0.92556...","[[-0.82639194, -0.82639194, -0.82639194, -0.82...","[[-0.73914635, -0.73914635, -0.73914635, -0.57...","[[-0.5983442, -0.5983442, -0.5983442, 0.034376...","[[-1.045434, -1.045434, -1.045434, -0.5077677,..."
labels,"[50258, 50259, 50359, 50363, 3322, 9488, 2610,...","[50258, 50259, 50359, 50363, 1902, 292, 1152, ...","[50258, 50259, 50359, 50363, 4286, 360, 561, 8...","[50258, 50259, 50359, 50363, 3322, 3237, 3031,...","[50258, 50259, 50359, 50363, 306, 12674, 6869,...","[50258, 50259, 50359, 50363, 72, 390, 13856, 4...","[50258, 50259, 50359, 50363, 675, 2198, 257, 2...","[50258, 50259, 50359, 50363, 546, 538, 472, 26...","[50258, 50259, 50359, 50363, 72, 500, 380, 458...","[50258, 50259, 50359, 50363, 3322, 707, 1594, ...",...,"[50258, 50259, 50359, 50363, 270, 632, 257, 14...","[50258, 50259, 50359, 50363, 2994, 412, 300, 5...","[50258, 50259, 50359, 50363, 5616, 458, 689, 7...","[50258, 50259, 50359, 50363, 13162, 6576, 281,...","[50258, 50259, 50359, 50363, 3322, 3295, 1194,...","[50258, 50259, 50359, 50363, 3322, 3237, 5694,...","[50258, 50259, 50359, 50363, 266, 527, 636, 28...","[50258, 50259, 50359, 50363, 71, 3145, 732, 29...","[50258, 50259, 50359, 50363, 20579, 456, 415, ...","[50258, 50259, 50359, 50363, 71, 3159, 82, 295..."


## Model Selection

In [16]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [17]:
model.generation_config.language = "english"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [18]:
model.config.dropout = 0.1

In [19]:
import torch
import numpy
from evaluate import load

def map_to_pred(batch):
    input_features = batch["input_features"]
    input_features = torch.from_numpy(input_features).float()
    input_features = input_features.unsqueeze(0)
    labels = batch["labels"]
    labels[labels == -100] = processor.tokenizer.pad_token_id
    reference = processor.tokenizer.decode(labels, skip_special_tokens=True)
    batch["reference"] = processor.tokenizer._normalize(reference)
    predicted_ids = model.to("cuda").generate(input_features.to("cuda"))[0]
    prediction = processor.tokenizer.decode(predicted_ids, skip_special_tokens=True)
    batch["prediction"] = processor.tokenizer._normalize(prediction)
    return batch

result = benchmark.apply(map_to_pred)

wer = load("wer")
print(100 * wer.compute(references=result.loc["reference"], predictions=result.loc["prediction"]))

C:\Users\lim_y\anaconda3\Lib\site-packages\transformers\models\whisper\tokenization_whisper.py:501: UserWarning: The private method `_normalize` is deprecated and will be removed in v5 of Transformers.You can normalize an input string using the Whisper English normalizer using the `normalize` method.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
C:\Users\lim_y\anaconda3\Lib\site-packages\transformers\models\whisper\modeling_whisper.py:697: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cu

4.732254047322541


In [20]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [21]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

## Model Evaluation

In [22]:
import evaluate

metric = evaluate.load("wer")

In [28]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    with open('refs_and_preds.txt', 'w') as f:
        for ref, pred in zip(label_str, pred_str):
            f.write(f"Ref: {ref}\n")
            f.write(f"Pred: {pred}\n\n")
    
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

## Model Training

In [24]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-test",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=5e-8,
    warmup_steps=250,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=400,
    eval_steps=400,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

C:\Users\lim_y\anaconda3\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train,
    eval_dataset=test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.tokenizer,
)

max_steps is given, it will override any value given in num_train_epochs


In [26]:
processor.save_pretrained(training_args.output_dir)

[]

In [30]:
trainer.train()

C:\Users\lim_y\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
400,0.004500,0.920914,30.769231
800,0.000800,0.898984,15.384615
1200,0.000300,0.895707,15.384615
1600,0.000200,0.893053,15.384615
2000,0.000100,0.892653,15.384615
2400,0.000100,0.892668,15.384615
2800,0.000100,0.891868,15.384615
3200,0.000100,0.891186,15.384615
3600,0.000100,0.891778,15.384615
4000,0.000100,0.892117,15.384615


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
C:\Users\lim_y\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpo

TrainOutput(global_step=4000, training_loss=0.00693877083962434, metrics={'train_runtime': 1263.5001, 'train_samples_per_second': 101.306, 'train_steps_per_second': 3.166, 'total_flos': 2.30868320256e+18, 'train_loss': 0.00693877083962434, 'epoch': 4000.0})

In [31]:
result = benchmark.apply(map_to_pred)

wer = load("wer")
print(100 * wer.compute(references=result.loc["reference"], predictions=result.loc["prediction"]))

C:\Users\lim_y\anaconda3\Lib\site-packages\transformers\models\whisper\tokenization_whisper.py:501: UserWarning: The private method `_normalize` is deprecated and will be removed in v5 of Transformers.You can normalize an input string using the Whisper English normalizer using the `normalize` method.
  warnings.warn(
C:\Users\lim_y\anaconda3\Lib\site-packages\transformers\models\whisper\tokenization_whisper.py:501: UserWarning: The private method `_normalize` is deprecated and will be removed in v5 of Transformers.You can normalize an input string using the Whisper English normalizer using the `normalize` method.
  warnings.warn(
C:\Users\lim_y\anaconda3\Lib\site-packages\transformers\models\whisper\tokenization_whisper.py:501: UserWarning: The private method `_normalize` is deprecated and will be removed in v5 of Transformers.You can normalize an input string using the Whisper English normalizer using the `normalize` method.
  warnings.warn(
C:\Users\lim_y\anaconda3\Lib\site-packages\

4.60772104607721


In [10]:
pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [32]:
from huggingface_hub import notebook_login

notebook_login()

In [33]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: en, split: test",
    "language": "en",
    "model_name": "Whisper Small Refined - Seagate Lim",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [34]:
trainer.push_to_hub(**kwargs)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

events.out.tfevents.1723277552.LAPTOP-F12I3AC6.972.0:   0%|          | 0.00/9.97k [00:00<?, ?B/s]

events.out.tfevents.1723277729.LAPTOP-F12I3AC6.972.1:   0%|          | 0.00/43.9k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/PaidDatasetsBad/whisper-small-test/commit/4dae8ff56912d90eae7108526468d10b017fe070', commit_message='End of training', commit_description='', oid='4dae8ff56912d90eae7108526468d10b017fe070', pr_url=None, pr_revision=None, pr_num=None)